<a href="https://colab.research.google.com/github/hyeonji0401/CUDA_practice/blob/main/thread_hierarchy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 4-1. CUDA 스레드 계층 구조

**1. CUDA 스레드 계층**
- 스레드 -> 워프 -> 블록 -> 그리드로 이루어짐

1) 스레드
- 계층 구조에서 가장 작은 단위
- CUDA 연산을 수행하거나 CUDA 코어르 사용하는 기본 단위
- 커널 코드는 모든 스레드에 공우되며 각 스레드가 독립적으로 커널 코드를 수행

2) 워프
- 계층 구조의 두번째 계층
- 32개의 스레드를 하나로 묶은 것을 말하며 CUDA의 기본 수행단위
- **기본수행단위 :** 한 워프에 속한 스레드들은 하나의 제어 장치에 의해 제어됨
- GPU의 SIMT구조에서 멀티 스레드의 단위가 되는 것이 워프임
=> 즉 하나의 명령에 따라 32개의 스레드가 동시에 움직임

3) 블록(스레드 블록)
- 워프보다 상위 스레드 그룹으로 워프들의 집합을 의미함
- 하나의 블록에 포함된 각 스레드는 자신만의 고유한 스레드 번호(thread ID)를 가짐

  : 동일한 블록 안에는 동일한 번호를 갖는 스레드가 없음
- 각 블록은 자신만의 고유한 블록 번호(block ID)가 있음

=> 원하는 스레드를 지정하기 위해서는 블록 번호와 스레드 번호를 모두 사용해야함

  : 블록 내 스레드는 최대 3차원 번호를 가질 수 있음

4) 그리드
- 계층 구조에서 가장 상위 단계
- 여러 개의 블록을 포함하는 블록들의 그룹
- 하나의 그리드에 포함된 블록들은 서로 다른 자신만의 번호를 가지며 그리드 내 블록 또한 최대 3차원 형태로 배치될 수 있음
- 커널이 호츨되면 그리드가 생성됨

=> 하나의 그리드는 하나의 커널 호출과 1:1 대응되게 해당 커널을 수행한 스레드를 생성함


**2. CUDA 스레드 계층을 위한 내장 변수들**

내장 변수 : 현재 그리드 및 블록의 형태와 각 스레드가 자신이 속한 블록 번호, 그리고 사진의 스레드 번호를 확인할 수 있음
> 스레드들이 자신이 처리할 데이터가 무엇인지 알기 위해서는 자신이 어떤 블록에 속해 있는지, 블록 내 자신의 스레드 번호를 알아야함

> 내장 변수의 값은 커널이 수행될 때 결정되면 각 스레드는 자신에게 할당된 내장 변수 값을 참조할 수 있음

> 내장 변수 값을 코드에서 수정할 수는 없음

**가. grimDim**

- 그리드 형태 정보를 담고 있는 구조체 내장 변수
- x,y,z멤버가 각각 첫번쨰 차원부터 세번째 차원의 크기를 담고 있음
- 각 차원의 길이는 1이상 양의 정수로 정의되어야하며, 소수점 값을 가질 수 없음
- 사용하지 않는 차원의 크기는 1로 표현됨
- 커널 내 모든 스레드가 공유함

   ex) x=3, y=2, z=1 : 3x2 크기의 2차원 형태로 이루어져있음

**나. blockIdx**

- 현재 스레드가 속한 블록의 번호를 담고 있는 구조체형 내장 변수
- 블록 번호는 0번부터 시작함
- 한 블록에 속한 모든 스레드들이 모두 공유하게 됨
  ex) blockIdx.x =2, blockIdx=2, blockIdx=0 : 2X2 크기의 2차원 형태로 이루어져있음

**다. blockDim**
- 블록의 형태 정보를 담고 있는 구조체형 내장 변수
- 각 차원의 길이는 1이상의 양의 정수로 정의되어야하며 소수점 값을 가질 수 없음
- 그리드 내 모든 스레드가 공유하게 됨

**라. threadIdx**
- 블록 내에서 현재 스레드가 부여받은 스레드 번호를 담고 있는 구조체형 내장 변수
- 0부터 시작하며 최대 3차원의 번호를 가질 수 있음
  ex) threadIdx.x = 2, threadIdx.y=2, threadIdx.z=0 : 2x2에 위치함

*** 스레드 번호와 워프의 구성**
> 워프는 연속된 32개의 스레드로 구성됨

> 스레드의 연속성은 threadIdx.x, y, z차원 순으로 결정됨

[즉 (0,0,0)~(31,0,0)번 스레드가 하나의 워프를 구성함]

> 커널의 성능에 큰 영향을 미치는 요소 중 하나인 메모리 접근 패턴을 이해하려면 워프를 구성하는 스레드를 정확히 인지해야함

> 워프 수준에서 스레드들 사이의 작업을 분배할 때도 중요한 역할을 함
